UC3MAL201 Machine Learning Session 11 Tutorial 11 Leon Eriksen Helgeland -
First written 09.09.2020

# Neural Network - with Keras

In [14]:
%%time
import numpy as np
#import tensorflow as tf
#from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow.keras.optimizers
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
import pandas as pd
from tensorflow.keras.utils import to_categorical # OneHotEncoder

Wall time: 0 ns


## 1. Dataset description

- Name of the datset: MNIST
- Contents:     28x28 pixel grayscale images
- Images contain: Handwritten single digits, 0-9
- Total dataset length: 70,000 images. 60,000 train-set and 10,000 test-set 
- Total dataset row length: 785 columns.
- First column: 1 class column, digit 0-9.
- All other columns : 784 feature columns, pixels from 0 to 255

## 2. Data pre-processing

In [2]:
%%time
# Import the dataset into train and test X and y
train = pd.read_csv(r'C:\Users\leon.helgeland\OneDrive\S05-MAL-2020\S05MAL - Session 11 - Neural Network with Keras - Live Demo\Dataset for the tutorial-20200908\mnist_train.csv',header=None)
test = pd.read_csv(r'C:\Users\leon.helgeland\OneDrive\S05-MAL-2020\S05MAL - Session 11 - Neural Network with Keras - Live Demo\Dataset for the tutorial-20200908\mnist_test.csv',header=None)
X_test = test.iloc[:,1:].apply(lambda a: a*(0.99/255)+0.01) # Rescaling 0-255 pixel values to 0.01-1
X_train = train.iloc[:,1:].apply(lambda a: a*(0.99/255)+0.01) # Rescaling 0-255 pixel values to 0.01-1
# Above: This rescaling process helps to prevent having 0 values as inputs which can cause weight update problems during network synthesis.

enc = OneHotEncoder(categories="auto", sparse=False, dtype=np.int16) # Removed: handle_unknown='ignore'
y_train = enc.fit_transform(train.iloc[:,:1].to_numpy().reshape(-1, 1)) # OneHotEncoding single column to binary columns
y_test = enc.fit_transform(test.iloc[:,:1].to_numpy().reshape(-1, 1)) # OneHotEncoding single column to binary columns

#tes = to_categoricaltrain.iloc[:,:1].to_numpy()#.to_numpy().reshape(-1, 1)
# Alternative way to try for the future work with keras
#y_train = to_categorical(train.iloc[:,:1].to_numpy().reshape(-1, 1),3) # 3 is number of classes
#y_test = to_categorical(test.iloc[:,:1], 3)

Wall time: 3.75 s


## 3. Modelling process

In [16]:
%%time
# Define model
def model_fit(X_train, y_train, config):
    n_input = config[0]
    n_output = config[1]
    n_nodes = config[2]
    epoch = config[3]
    batch = config[4]
    # Define your model - Create sequential model and add configured layers.
    model = Sequential() # Model activation
    model.add(Dense(n_nodes, input_dim=n_input, activation='relu')) # Hidden layer with 'relu' function. Also the shape of input is also set, which sets up the input layer automatically.
    # model.add(Dropout(0.2)) # tried activating dropout layer 0.2 which interestingly kept the loss at atleast 0.200 so maybe there is a connection here. Anyway the result wasy worse
    # model.add(Dense(8, activation='relu')) # Hidden layer with 'relu' function
    # model.add(Dropout(0.2)) 
    # Test adding these layers:
    # - Conv + Pool x2
    # - Conv x2 (optional)
    # - Conv + Pool x1 (optional)
    # - Full x2
    # - Try dropout
    model.add(Dense(n_output, activation='softmax')) # Output layer with 'softmax' function 
    # Compile your model - Specify loss function and optimizers and call the compile() function on the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # Compiling the output # Metrics does not work when doing regression since it can only use mean square error
    # Fit your model - Train the model on a sample of data by calling the fit() function on the model
    model.fit(X_train, y_train, epochs=epoch, batch_size=batch) # A batch size of 1 will load one row at the time. A higher batch size will reduce training time. When doin millions of rows the batch size should be as large as possible.But when using a small dataset its good to have a smaller batch size so that the model can learn something from each row.
    return model

Wall time: 0 ns


In [4]:
%%time
# Make predictions - Use the model to generate predictions on new data by calling functions such as evaluate() or predict() on the model.

# predict with the fit model
def model_predict(model, testdata):
    Prediction = model.predict(testdata, verbose=0)
    return Prediction

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

Wall time: 0 ns


## 4. Train your model

In [17]:
#config = [int(len(X_train[0])), int(len(y_train[0])), 20, 100, 1] # n_inputs, n_outputs, n_nodes(hidden_layer), epoch, batch
config = [784, 10, 20, 100, 10] # n_inputs, n_outputs, n_nodes(hidden_layer), epoch, batch
neural_network = model_fit(X_train, y_train, config)
# 1. loss: 0.0326 - accuracy: 0.9893
# 2. dropout layer active: loss: 0.2165 - accuracy: 0.9298

Epoch 1/100
6000/6000 [==============================] - 3s 513us/step - loss: 0.5548 - accuracy: 0.8410
Epoch 2/100
6000/6000 [==============================] - 3s 503us/step - loss: 0.1916 - accuracy: 0.9456
Epoch 3/100
6000/6000 [==============================] - 3s 502us/step - loss: 0.1569 - accuracy: 0.9546
Epoch 4/100
6000/6000 [==============================] - 3s 507us/step - loss: 0.1347 - accuracy: 0.9606
Epoch 5/100
6000/6000 [==============================] - 3s 507us/step - loss: 0.1242 - accuracy: 0.9639
Epoch 6/100
6000/6000 [==============================] - 3s 506us/step - loss: 0.1192 - accuracy: 0.9647
Epoch 7/100
6000/6000 [==============================] - 3s 509us/step - loss: 0.1112 - accuracy: 0.9672
Epoch 8/100
6000/6000 [==============================] - 3s 506us/step - loss: 0.1098 - accuracy: 0.9679
Epoch 9/100
6000/6000 [==============================] - 3s 503us/step - loss: 0.0996 - accuracy: 0.9701
Epoch 10/100
6000/6000 [==============================]

## 5. Prediction process

In [6]:
prediction = model_predict(neural_network, X_test)
prediction

array([[1.2517196e-19, 6.4163129e-22, 5.1322519e-11, ..., 9.9864155e-01,
        6.3047820e-09, 6.7824821e-08],
       [8.5671092e-22, 8.5218860e-18, 1.0000000e+00, ..., 4.7335407e-35,
        4.2230999e-14, 0.0000000e+00],
       [6.3037625e-20, 9.9997032e-01, 1.6900708e-06, ..., 3.5386438e-07,
        2.7707645e-05, 7.6585224e-11],
       ...,
       [9.9224546e-35, 3.3007824e-23, 3.4325831e-24, ..., 2.0167606e-13,
        1.0487506e-14, 7.3828545e-07],
       [5.8037053e-20, 4.4532130e-22, 5.3326466e-29, ..., 3.8933756e-21,
        1.1240726e-08, 3.2352341e-18],
       [1.2681082e-22, 2.1130720e-29, 8.4139941e-26, ..., 0.0000000e+00,
        8.5284289e-24, 2.2106506e-27]], dtype=float32)

In [11]:
# Evaluate the model
too_bad = list() # My own

PredClass = list()
ActualClass = list()
for row in range(len(y_test)):
    index_maxscore1 = max(range(len(y_test[row])), key=y_test[row].__getitem__)
    ActualClass.append(index_maxscore1)
    index_maxscore2 = max(range(len(prediction[row])), key=prediction[row].__getitem__)
    PredClass.append(index_maxscore2)
    if index_maxscore1 != index_maxscore2: # My own
        too_bad.append([row,index_maxscore1,index_maxscore2]) # My own
    print('Expected=%d, Got=%d' % (index_maxscore1, index_maxscore2))
print('Number of errors:', len(too_bad), 'of', len(y_test)) # My own


accuracy = accuracy_metric(PredClass, ActualClass)
accuracy
# 1: Number of errors 474 of 10000 , 95.26

, Got=6
Expected=7, Got=7
Expected=1, Got=1
Expected=9, Got=9
Expected=8, Got=8
Expected=0, Got=0
Expected=6, Got=6
Expected=9, Got=9
Expected=4, Got=4
Expected=9, Got=9
Expected=9, Got=9
Expected=6, Got=6
Expected=2, Got=2
Expected=3, Got=3
Expected=7, Got=7
Expected=1, Got=1
Expected=9, Got=9
Expected=2, Got=2
Expected=2, Got=2
Expected=5, Got=5
Expected=3, Got=3
Expected=7, Got=7
Expected=8, Got=8
Expected=0, Got=0
Expected=1, Got=1
Expected=2, Got=2
Expected=3, Got=3
Expected=4, Got=4
Expected=7, Got=7
Expected=8, Got=8
Expected=9, Got=9
Expected=0, Got=0
Expected=1, Got=1
Expected=2, Got=2
Expected=3, Got=3
Expected=4, Got=4
Expected=7, Got=7
Expected=8, Got=8
Expected=9, Got=9
Expected=0, Got=0
Expected=1, Got=1
Expected=7, Got=7
Expected=8, Got=8
Expected=9, Got=9
Expected=8, Got=8
Expected=9, Got=9
Expected=2, Got=2
Expected=6, Got=6
Expected=1, Got=1
Expected=3, Got=3
Expected=5, Got=5
Expected=4, Got=4
Expected=8, Got=8
Expected=2, Got=2
Expected=6, Got=6
Expected=4, Got=4
Ex

95.26

## 6. K-fold Cross Validation

In [21]:
# Making a list of configurations to try.
def model_configs():
    # Defining the scopes of the configs
    n_nodes = [10, 20, 25]
    n_epochs = [100, 200, 500]
    n_batch = [5, 10, 50]
    # Create the configs
    configs = list()
    for i in n_nodes:
        for j in n_epochs:
            for k in n_batch:
                cfg = [784, 10, i, j, k]
                configs.append(cfg)
    print(f'Total amount of configurations: {len(configs)}')
    return configs

In [1]:
cfg_list = model_configs()
cfg_list

cfg_scores = []

# Performing K-fold cross validation
for cfg in cfg_list:
    neural_network = model_fit(X_train, y_train, cfg)

NameError: name 'model_configs' is not defined

In [ ]:
cfg_list

# Testing

In [ ]:
df = pd.read_csv('dataset/')
def rescale_image(a)
    '''
    Takes a image value between 0-255 as input and converts it to a value between 0.01 and 1.
    '''
    return lambda a : a*(0,99/255)+0.01
df[:-1] = [[rescale_image(pixel) for pixel in row] for row in df[:-1]]

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import KFold
import tensorflow.keras.optimizers
import os
import tensorflow
print(tensorflow.__version__)

In [ ]:
%%time
dataset_train_file = 'mnist_train.csv'
dataset_test_file = 'mnist_test.csv'
dataset_directory = r'C:\Users\leon.helgeland\OneDrive\S05-MAL-2020\S05MAL - Session 11 - Neural Network with Keras - Live Demo\Dataset for the tutorial-20200908'
#'~/GitHub/BigData/dataset11'
# import train and test dataset
train = pd.read_csv(f'{dataset_directory}/{dataset_train_file}',header=None)
test = pd.read_csv(f'{dataset_directory}/{dataset_test_file}',header=None)
# Import dataset
X_train = train.iloc[:,1:].apply(lambda a: a*(0.99/255)+0.01)
y_train = train.iloc[:,:1]
X_test = test.iloc[:,1:].apply(lambda a: a*(0.99/255)+0.01)
y_test = test.iloc[:,:1]
# The class labels are integer values, encoding them into one-hot representation https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(categories="auto", sparse=False, dtype=np.int16) # handle_unknown='ignore'
y_train = enc.fit_transform(y_train.to_numpy().reshape(-1, 1))
y_test = enc.fit_transform(y_test.to_numpy().reshape(-1, 1))

## Dataset preparation
Rescale the data to values between 0.01 and 1, by multiplying each pixel value in the dataset by (0.99/255) and adding 0.01 to the result to get a new pixel value.

In [ ]:
def model_fit(X_train, y_train, config):
    n_input = config[0]
    n_output = config[1]
    n_nodes = config[2]
    epoch = config[3]
    batch = config[4]
    # Define and Compile
    model = Sequential()
    model.add(Dense(n_nodes, input_dim=n_input, activation='relu'))
    model.add(Dense(n_output, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # Lossfunction is low when the percentage prediction of each row is a class is close to the correct class(ex 0.9 of 1.0?) and high when far away ex 0.52 of 1.0)
    # Fit the model
    model.fit(X_train, y_train, epochs=epoch, batch_size=batch)
    return model

# predict with the fit model
def model_predict(model, testdata):
    Prediction = model.predict(testdata, verbose=0)
    return Prediction

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [ ]:
inputs = 784 # Input features
outputs = 10 # Output features
nodes = [5,10,15,20,35,50] # Test amount of nodes in layer

configs = []
for n in nodes:
    config_ = [inputs, outputs, n, 75, 250]
    configs.append(config_)

In [ ]:
# Kfolding
config_lossess = []
accuracy_per_config = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)

counter = 0
for config in configs:
    counter+=1
    fold_count = 0

    fold_losses = []
    fold_accuracy = []
    
    for fold_train, fold_test in kf.split(X_train, y_train):
        fold_count+=1
        nn_model = model_fit(X_train[fold_train], y_train[fold_train], config)
        scores = neural_network_model.evaluate(X_train[fold_test], y_train[fold_test], verbose=0)
        print(f"Score for config {counter} fold {fold_count}: {neural_network_model.metrics_names[0]}={scores[0]}; {neural_network_model.metrics_names[1]}={scores[1]*100}%")
        
        fold_losses.append(scores[0])
        fold_accuracy.append(scores[1]*100)

    config_lossess.append(fold_losses)
    accuracy_per_config.append(fold_accuracy)

In [ ]:
print("Score per config: ")
for j in range(0, len(accuracy_per_config)):
    print('^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
    print(f'Score for config {j+1}. Nodes:{configs[j][2]}')
    print('Average scores for all folds:')
    print(f'> Loss: {np.mean(config_lossess[j])}')
    print(f'> Accuracy: {np.mean(accuracy_per_config[j])} (+- {np.std(accuracy_per_config[j])})')
    print('')

In [ ]:
configy = [784, 10, 20, 10, 1]
neural_network = model_fit(X_train, y_train, configy)

In [ ]:
# Evaluate the model
prediction = model_predict(neural_network, X_test)

PredClass = list()
ActualClass = list()
for row in range(len(y_test)):
    index_maxscore1 = max(range(len(y_test[row])), key=y_test[row].__getitem__)
    ActualClass.append(index_maxscore1)
    index_maxscore2 = max(range(len(prediction[row])), key=prediction[row].__getitem__)
    PredClass.append(index_maxscore2)
    print('Expected=%d, Got=%d' % (index_maxscore1, index_maxscore2))


accuracy = accuracy_metric(PredClass, ActualClass)
accuracy

In [2]:
    import tensorflow as tf
    if tf.test.gpu_device_name():
        print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
    else:
        print("Please install GPU version of TF")

Please install GPU version of TF
